In [ ]:
# === P-LBAL — Umbral por percentiles de balance después de IN Cash =============
# Lógica de la REGLA:
#   tx_direction = Inbound
#   AND tx_type = Cash
#   AND (customer_base_account_balance [default 0] + tx_base_amount) > Balance
#
# Parametrización propuesta:
#   Balance = percentil alto (p90/p95/p97/p99) de la distribución de
#             balance_after = max(customer_base_account_balance, 0 si NaN) + |tx_base_amount|
#             evaluado sobre todas las transacciones Inbound Cash.

import pandas as pd
import numpy as np

# -------- Parámetros editables --------
PATH = "../../data/tx/datos_trx__with_subsub.csv"
SUBSUBSEGMENTS = "I-2"   # <-- ajusta el sub-subsegmento
PCTS = [95, 97, 99]

# -------- Carga mínima --------
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")

# -------- Filtro: Inbound + Cash --------
df["tx_date_time"]   = pd.to_datetime(df.get("tx_date_time"), errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df.get("tx_base_amount"), errors="coerce")
df["tx_direction"]   = df.get("tx_direction", "").astype(str).str.title()
df["tx_type"]        = df.get("tx_type", "").astype(str).str.title()

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

mask = (
    df["tx_direction"].eq("Inbound") &
    df["tx_type"].eq("Cash") &
    df["tx_base_amount"].notna()
)
g = df.loc[mask, ["customer_id", "tx_date_time", "customer_account_balance", "tx_base_amount"]].copy()

if g.empty:
    print("No hay transacciones Inbound Cash elegibles para P-LBAL.")
else:
    # Defaults y normalización:
    # - Balance previo en BASE: default 0 si viene NaN
    # - Monto de la tx en BASE: usamos absoluto por robustez de signos contables
    g["customer_account_balance"] = pd.to_numeric(g["customer_account_balance"], errors="coerce").fillna(0.0)
    g["tx_base_amount"] = g["tx_base_amount"].abs()

    # Balance después de la transacción (definición de la regla)
    g["balance_after_tx"] = g["customer_account_balance"] + g["tx_base_amount"]

    # Percentiles y sugerencia
    s = g["balance_after_tx"].astype(float).dropna()
    if s.empty:
        print("No se pudo construir la distribución de 'balance_after_tx'.")
    else:
        stats = {f"p{p}": float(np.percentile(s, p)) for p in PCTS}
        # Elige aquí el percentil que usarás como propuesta (ej. p95)
        recommended = int(round(stats["p95"])) if np.isfinite(stats["p95"]) else np.nan

        print("=== P-LBAL — Percentiles de balance_after_tx (CLP, por transacción IN Cash) ===")
        print(f"Transacciones consideradas: {len(s):,}")
        for p in PCTS:
            print(f"p{p:>2}: {stats[f'p{p}']:,.0f}")
        print(f"\nSugerencia de Balance (p95): {recommended:,.0f} CLP")

        # (Opcional) Cobertura: % de eventos que gatillarían con ese umbral
        coverage = (s > recommended).mean() * 100 if np.isfinite(recommended) else np.nan
        print(f"Cobertura estimada a p95: {coverage:.2f}% de eventos IN Cash")


C:\Users\vinso\AppData\Local\Temp\ipykernel_1480\1618149578.py:21: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== P-LBAL — Percentiles de balance_after_tx (CLP, por transacción IN Cash) ===
Transacciones consideradas: 585
p95: 4,725,306,688
p97: 5,778,638,275
p99: 7,767,207,717

Sugerencia de Balance (p95): 4,725,306,688 CLP
Cobertura estimada a p95: 5.13% de eventos IN Cash


# Simulación alertas

In [6]:
# === P-LBAL — Sensibilidad (Actual vs propuestos) ==============================
# LÓGICA EXACTA:
# tx_direction = Inbound
# AND customer_base_account_balance [Default: 0] + tx_base_amount > [Balance]
# Unidad = transacciones que cumplen

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = ["I-1"]               # <-- ajusta el sub-subsegmento
PARAMS = {
    "Actual": {"Balance": 1_206_067_841},
    "p95":    {"Balance": 777_169_430},
    "p97":    {"Balance": 1_083_773_658},
    "p99":    {"Balance": 2_609_110_063},
}

df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_direction"]=df["tx_direction"].astype(str).str.title()
df["tx_type"]=df["tx_type"].astype(str).str.title()
df["tx_base_amount"]=pd.to_numeric(df["tx_base_amount"], errors="coerce")
bal = pd.to_numeric(df["customer_account_balance"], errors="coerce").fillna(0)
df["_bal_prev"] = bal

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

g = df[(df["tx_direction"].eq("Inbound")) & df["tx_base_amount"].notna()].copy()

order=["Actual","p90","p95","p96","p97","p99"]
param_tbl=(pd.DataFrame(PARAMS).T.loc[[k for k in order if k in PARAMS]]
           .rename_axis("escenario").reset_index())
print("=== P-LBAL — Parámetros ==="); display(param_tbl)

counts={}
for k,v in PARAMS.items():
    B = v["Balance"]
    counts[k]=int((g["_bal_prev"] + g["tx_base_amount"] > B).sum())

out=pd.DataFrame([{
    "alertas_actual":counts.get("Actual",0),
    "alertas_p90":counts.get("p90",0),
    "alertas_p95":counts.get("p95",0),
    "alertas_p96":counts.get("p96",0),
    "alertas_p97":counts.get("p97",0),
    "alertas_p99":counts.get("p99",0),
}])
print("=== P-LBAL — Alertas por escenario (tx) ==="); display(out)


C:\Users\vinso\AppData\Local\Temp\ipykernel_22700\1180188902.py:19: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== P-LBAL — Parámetros ===


,escenario,Balance
0,Actual,1206067841
1,p95,777169430
2,p97,1083773658
3,p99,2609110063


=== P-LBAL — Alertas por escenario (tx) ===


,alertas_actual,alertas_p90,alertas_p95,alertas_p96,alertas_p97,alertas_p99
0,187,0,361,0,217,73
